# Практические задание №3

Ваша задача -- определить тональность текста по шкале от 1 (негативный) до 10 (позитивный).
Ввод: тексты, разделенные переводом строки (\n).
Вывод: для каждого текста из входных данных вывести тональную оценку от 1 до 10. Разделитель между выводами для разных текстов -- перевод строки (\n).

Для обучения можно использовать коллекцию текстов и соответствующие им оценки. Файл обучающей коллекции соответствует формату ввода, файл с оценками -- формату вывода; кодировка -- UTF-8.

В качестве оценки используется квадратный корень из среднеквадратической ошибки (Root Mean Squared Error) .

Вы можете кратко описать решение в первой строке загружаемого файла после символа #. Информация будет полезна авторам курса, чтобы составить представление об используемых методах и подходах.

In [46]:
# Загрузка необходимых библиотек 

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline 
from sklearn.metrics import mean_squared_error 
from sklearn.model_selection import train_test_split 
from sklearn.feature_extraction.text import TfidfTransformer 
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.naive_bayes import MultinomialNB 
from natasha import MorphVocab, Segmenter, NewsEmbedding, NewsMorphTagger, NewsSyntaxParser, NewsNERTagger, Doc 

In [39]:
# Загрузим данные из файла `texts_train.txt`, посмотрим на них
import csv 

texts_train = pd.read_csv(
    'C:/Users/Григорий/Notebooks (Pyton)/Обработка_языка/Практическое_задание_3/texts_train.txt', 
    sep='\n', header=None, encoding='utf-8', error_bad_lines=False, quoting=csv.QUOTE_NONE) 
    '''Так как поля содержат кавычки, чтобы не возникало ошибок, 
       не будем обрабатывать: quoting=csv.QUOTE_NONE'''
texts_train.head() 

,0
0,"Сериал очень люблю, но Академия и Земля вызыва..."
1,"думал, что будет лучше идея очень интересна - ..."
2,с творчеством Головачева я познакомился посред...
3,"то-то я и в большое неудовольствие прочитал ""А..."
4,как мне показалось местами сильно смахивает на...


In [40]:
texts_train.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       20000 non-null  object
dtypes: object(1)
memory usage: 156.4+ KB


In [9]:
# Загрузим данные из файла `scores_train.txt`, посмотрим на них

scores_train = pd.read_csv(
    'C:/Users/Григорий/Notebooks (Pyton)/Обработка_языка/Практическое_задание_3/scores_train.txt', 
    sep="\n", header=None, error_bad_lines=False) 
scores_train.head() 

,0
0,6
1,7
2,10
3,5
4,6


In [10]:
scores_train.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   0       20000 non-null  int64
dtypes: int64(1)
memory usage: 156.4 KB


Как видим, количество строк одинаковое, теперь можем записать наши данные в параметры и целевую переменную:

In [53]:
X = texts_train[0] 
y = scores_train 

Учтем, что у нас входные данные на русском языке и слова имеют разные окончания. Приведем их к одному виду с помощью лемматизации, а также используем результаты синтаксического разбора, учтем связи между словами, нормализуем именованные сущности. В этом нам поможет `Natasha`.

In [42]:
emb = NewsEmbedding() 
morph_vocab = MorphVocab() 
segmenter = Segmenter() 
morph_tagger = NewsMorphTagger(emb) 
syntax_parser = NewsSyntaxParser(emb) 
ner_tagger = NewsNERTagger(emb) 

# Функция для получения списка токенов для русского языка

def docVocab(text): 
    doc = Doc(text) 
    doc.segment(segmenter) 
    doc.tag_morph(morph_tagger) 
    
    for token in doc.tokens:
        token.lemmatize(morph_vocab) 
        
    lemms = {_.text: _.lemma for _ in doc.tokens if _.pos != 'PUNCT'} 
    
    doc.parse_syntax(syntax_parser) 
    doc.tag_ner(ner_tagger) 
    
    for span in doc.spans:
        span.normalize(morph_vocab)
    
    spans = {_.text: _.normal for _ in doc.spans} 
    lemms.update(spans) 
    
    return list(lemms.values()) 

In [54]:
# Применим функцию, написанную выше для предобработки входных данных, перед этим приведем в строки

X = X.apply(str) 
X_new = X.apply(lambda x: ' '.join(docVocab(x))) 

In [59]:
# Разобъем выборку предобработанных данных на обучающую и тестовую

X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(X_new, y[0], test_size=0.25) 

In [60]:
text_clf_new = Pipeline([('vector', CountVectorizer()), 
                       ('tfidf', TfidfTransformer()), 
                       ('clf', MultinomialNB())])

text_clf_new = text_clf_new.fit(X_train_new, y_train_new) 

In [61]:
y_predicted_new = text_clf_new.predict(X_test_new) 

In [64]:
# Root Mean Squared Error 

RMSE = (mean_squared_error(y_test_new, y_predicted_new))**0.5 
print(f'{RMSE:.2f}') 

2.88


Поищем параметры для получения более лучшего RMSE: 

In [66]:
from sklearn.model_selection import GridSearchCV 

parameters = {'vector__ngram_range': [(1, 1), (1, 2), (2, 2)], 
              'tfidf__use_idf': (True, False), 
              'clf__alpha': (1e-2, 1e-3)} 

In [76]:
text_clf = Pipeline([('vector', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())]) 
grid_search_clf = GridSearchCV(text_clf, parameters, scoring=mean_squared_error, n_jobs=-1)
grid_search_clf = grid_search_clf.fit(X_train_new, y_train_new) 

D:\Anaconda\lib\site-packages\sklearn\model_selection\_search.py:969: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


In [69]:
# Посмотрим на параметры

grid_search_clf.best_score_, grid_search_clf.best_params_

(0.30593333333333333,
 {'clf__alpha': 0.01, 'tfidf__use_idf': False, 'vector__ngram_range': (1, 2)})

In [71]:
# Функция для использования cross_val_score

def estimate_mse(clf, X, y, cv=5, scoring=mean_squared_error):
    return cross_val_score(clf, X, y, cv=5, scoring=mean_squared_error).mean() 

In [72]:
grid_search_param = grid_search_clf.best_params_ 

text_clf = Pipeline([('vector', CountVectorizer(ngram_range=grid_search_param['vector__ngram_range'])), 
                     ('tfidf', TfidfTransformer(use_idf=grid_search_param['tfidf__use_idf'])), 
                     ('clf', MultinomialNB(alpha=grid_search_param['clf__alpha']))]) 

In [81]:
from sklearn import svm 

text_clf_svm = Pipeline([('vector', CountVectorizer(ngram_range=(1, 2))), 
                         ('tfidf', TfidfTransformer(use_idf=False)), 
                         ('clf', svm.SVC())]) 
text_clf_svm.fit(X_train_new, y_train_new) 

Pipeline(steps=[('vector', CountVectorizer(ngram_range=(1, 2))),
                ('tfidf', TfidfTransformer(use_idf=False)), ('clf', SVC())])

In [82]:
y_pred_new = text_clf_svm.predict(X_test_new) 

In [83]:
# Root Mean Squared Error 

RMSE_svm = (mean_squared_error(y_test_new, y_pred_new))**0.5 
print(f'{RMSE_svm:.2f}') 

2.40


SVM показывает более хороший результат

In [84]:
# Обучим модель на всех данных: 

X = texts_train[0] 
y = scores_train[0] 
text_svm = Pipeline([('vector', CountVectorizer(ngram_range=(1, 2))), 
                    ('tfidf', TfidfTransformer(use_idf=False)), 
                    ('clf', svm.SVC())]) 
text_svm.fit(X, y) 

Pipeline(steps=[('vector', CountVectorizer(ngram_range=(1, 2))),
                ('tfidf', TfidfTransformer(use_idf=False)), ('clf', SVC())])

Теперь предскажем оценки на входных данных: 

In [90]:
# Загрузим данные из файла `dataset_40757_1.txt`, посмотрим на них

dataset = pd.read_csv(
    'C:/Users/Григорий/Notebooks (Pyton)/Обработка_языка/Практическое_задание_3/dataset_40757_1.txt', 
    sep='\n', header=None, encoding='utf-8', error_bad_lines=False, quoting=csv.QUOTE_NONE) 
'''Так как поля содержат кавычки, чтобы не возникало ошибок, 
                   не будем обрабатывать: quoting=csv.QUOTE_NONE'''
dataset.head() 

,0
0,"Нет, с ""Американской трагедией"" ни в какое сра..."
1,"Очень добрый, красивый и, несмотря на внешнюю ..."
2,Лёгонькая книжечка на тему истории семьи Бордж...
3,"Фильм хороший. Музыка, танцы. Особенно на выпу..."
4,Стоящее кино. Мне оно показалось чем-то между ...


In [92]:
# Применим функцию, написанную выше для предобработки входных данных, перед этим приведем в строки

X_dataset = dataset[0].apply(str) 
X_dataset = X_dataset.apply(lambda x: ' '.join(docVocab(x))) 

In [93]:
# Предскажем значения 

y_pred = text_svm.predict(X_dataset) 

In [96]:
# Сохраним файл `predictions.txt` 

pd.DataFrame(y_pred).to_csv('predictions.txt', sep='\n', header=None, index=None) 

In [94]:
y_pred

array([ 9,  9,  8,  8, 10,  8,  9, 10,  8,  9,  9,  9,  9, 10,  9,  8, 10,
       10,  8, 10, 10,  9, 10,  8, 10,  9,  8,  9,  8, 10,  9,  9,  9,  8,
       10, 10, 10, 10, 10,  9,  9, 10,  9,  9, 10, 10,  8, 10,  9, 10, 10,
        9, 10, 10,  8, 10,  9,  8, 10,  9, 10, 10,  9,  9,  9,  8,  8, 10,
        9, 10, 10,  8,  8, 10,  9, 10, 10,  9,  8,  9,  9,  9, 10, 10,  9,
        9,  8, 10,  9, 10, 10, 10,  8, 10,  9,  9,  9, 10, 10, 10, 10, 10,
       10,  8, 10,  9,  8, 10,  8,  9, 10, 10,  9, 10,  9,  9,  8,  9, 10,
        9,  8, 10, 10, 10,  9, 10, 10,  9,  9,  9,  8,  8,  9, 10,  9, 10,
        9,  8, 10, 10, 10,  9, 10, 10,  9,  8,  9, 10,  8,  9,  8, 10, 10,
       10,  9, 10,  9, 10,  7,  9,  9, 10, 10,  8, 10,  9,  8, 10,  9,  5,
       10,  9,  9,  9,  9, 10,  8, 10,  8,  9, 10, 10,  9, 10,  9,  4,  8,
        9,  8,  9,  8, 10, 10,  7, 10,  9, 10,  9, 10,  8,  9,  8, 10,  9,
        8,  8, 10,  9,  9,  9, 10, 10,  5, 10,  9, 10, 10,  9,  9,  9,  9,
       10, 10,  8, 10, 10

In [97]:
# Создаём объект, который будет токенизировать данные
analyzer = CountVectorizer().build_analyzer() 

# Токенизируем набор данных
docs = [] 
for document in X: 
    docs.append(analyzer(document.replace('_', ''))) 

In [100]:
X_analyzer = pd.Series([' '.join(doc) for doc in docs]) 

In [102]:
text_analyzer_svm = Pipeline([('vector', CountVectorizer(ngram_range=(1, 2))), 
                         ('tfidf', TfidfTransformer(use_idf=False)), 
                         ('clf', svm.SVC())]) 
text_analyzer_svm.fit(X_analyzer, y) 

Pipeline(steps=[('vector', CountVectorizer(ngram_range=(1, 2))),
                ('tfidf', TfidfTransformer(use_idf=False)), ('clf', SVC())])

Теперь предскажем оценки на входных данных c analyzer: 

In [104]:
# Загрузим данные из файла `dataset_40757_2.txt`, посмотрим на них

dataset2 = pd.read_csv(
    'C:/Users/Григорий/Notebooks (Pyton)/Обработка_языка/Практическое_задание_3/dataset_40757_2.txt', 
    sep='\n', header=None, encoding='utf-8', error_bad_lines=False, quoting=csv.QUOTE_NONE) 
'''Так как поля содержат кавычки, чтобы не возникало ошибок, 
                   не будем обрабатывать: quoting=csv.QUOTE_NONE'''
dataset2.head() 

,0
0,"Какой гад сказал мне, что ""Центурион"" - кино и..."
1,Да что делать с этими англичосами? Снять фильм...
2,И снова нас ждет пересказ событий MS Зетa Ганд...
3,"Даже странно, что Князь Серебряный не входит в..."
4,"Вообще-то книгу я уже давно читал, но, кажется..."


In [105]:
# Создаём объект, который будет токенизировать данные
analyzer = CountVectorizer().build_analyzer() 

# Токенизируем набор данных
docs2 = [] 
for document in dataset2[0]: 
    docs2.append(analyzer(document.replace('_', ''))) 

In [106]:
# Сохраним токенизированный текст в переменную: 

X_analyzer2 = pd.Series([' '.join(doc) for doc in docs2]) 

In [107]:
# Предскажем значения 

y_pred2 = text_analyzer_svm.predict(X_analyzer2) 

In [108]:
# Сохраним файл `predictions2.txt` 

pd.DataFrame(y_pred2).to_csv('predictions2.txt', sep='\n', header=None, index=None) 

Простая модель без Наташи показала лучший результат 65.42 балла.